## Phase 4: Integrate Generative AI

### Generative AI Overview:
**Generative AI** is a type of **artificial intelligence** that can produce original content—such as text, images, audio, or code—based on user prompts. It uses deep learning models that mimic human cognitive processes to detect patterns in massive datasets and generate relevant outputs in response to natural language inputs. [1]

### How can Generative AI enhance our system?
In this assignment, we integrate **Generative AI** into our pre-existing supervised skincare recommendation system using ChatGPT, a language model developed by OpenAI. [2]

ChatGPT (short for Chat Generative Pre-trained Transformer) processes user inputs like skin type and desired skincare effects to produce personalized product advice. This makes it a powerful tool for recommendation systems, where user-specific data can be interpreted in real time to generate tailored outputs without relying on fixed decision trees or manually curated product lists. By leveraging ChatGPT's natural language understanding capabilities, the system becomes more adaptive, scalable, and capable of engaging users in a conversational interface.

### Effectiveness of Prompts
The sentences or questions that you enter into a **Generative AI** tool, also known as prompts, greatly affect the quality of outputs you receive. After you enter a prompt, the model analyzes the input and generates a response based on the patterns it has learned through its training. Descriptive prompts can enhance the quality of the outputs. [3]

By experimenting with two different prompt templates, we explore how variations in prompt detail influence the quality and specificity of AI-generated recommendations. This allows us to compare and justify the effectiveness of each template in delivering clear, useful, and user-centered skincare suggestions.

**Prompt 1:** "You are a beauty advisor. Based on the user's skin type: {skin_type}, and desired effect: {notable_effect}, 
recommend a product type and explain why."

**Prompt 2:** "You are an expert skincare consultant. A user has a {skin_type} skin type and is looking for products with 
the following effect: {notable_effect}. Consider the price range around {price}, and explain in detail which 
product type you would recommend from the following options: {product_options}. Also explain how the product 
helps and why it suits the user's needs."

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

from openai import OpenAI  # NEW import style for v1+

load_dotenv()

# Create OpenAI client with your key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# Prompt Templates
template_1 = """
You are a beauty advisor. Based on the user's skin type: {skin_type}, and desired effect: {notable_effect}, 
recommend a product type and explain why.
"""

template_2 = """
You are an expert skincare consultant. A user has a {skin_type} skin type and is looking for products with 
the following effect: {notable_effect}. Consider the price range around {price}, and explain in detail which 
product type you would recommend from the following options: {product_options}. Also explain how the product 
helps and why it suits the user's needs.
"""

# User Input
skin_type = "Oily"
notable_effect = "Anti-acne"
price = "150 SAR"
product_options = ["Cleanser", "Serum", "Moisturizer", "Toner"]

# Format Prompts
prompt_1 = template_1.format(skin_type=skin_type, notable_effect=notable_effect)
prompt_2 = template_2.format(
    skin_type=skin_type,
    notable_effect=notable_effect,
    price=price,
    product_options=", ".join(product_options)
)

# Ask GPT using the new client
def ask_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4" if you have access
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content

# Get responses
response_1 = ask_gpt(prompt_1)
response_2 = ask_gpt(prompt_2)

# Display results
print("=== Template 1 Response ===\n", response_1)
print("\n=== Template 2 Response ===\n", response_2)


=== Template 1 Response ===
 I recommend a salicylic acid-based cleanser or toner for your oily skin and desired anti-acne effect. Salicylic acid is a beta hydroxy acid that can effectively penetrate into the pores to exfoliate and unclog them, reducing the occurrence of acne breakouts. It also helps to regulate oil production and prevent future breakouts. Using a salicylic acid-based product consistently can help to improve the overall appearance of your skin and minimize acne flare-ups.

=== Template 2 Response ===
 For someone with oily skin looking for anti-acne products within a price range of around 150 SAR, I would recommend a cleanser and a serum.

Cleanser: I would recommend a gel-based cleanser specifically formulated for oily and acne-prone skin. Look for ingredients like salicylic acid, tea tree oil, or benzoyl peroxide which can help unclog pores, reduce excess oil production, and target acne-causing bacteria. A good option within the price range could be a salicylic acid 

In [5]:
# === Imports ===
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from catboost import CatBoostClassifier
import ipywidgets as widgets
from IPython.display import display, HTML

# === Load API Key ===
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# === Load CatBoost model ===
model = CatBoostClassifier()
model.load_model("catboost_model.cbm")  

# ✅ Print feature names
print("✅ Model Feature Names:")
print(model.feature_names_)

# Load original dataframe to get product type labels
original_df = pd.read_csv("../Dataset/processed_file.csv")
clustered_df = pd.read_csv("../Dataset/clustered_data_k6.csv")
original_df["Cluster"] = clustered_df["Cluster"]

product_type_columns = [col for col in original_df.columns if col.startswith("product_type_")]
index_to_label = [col.replace("product_type_", "") for col in product_type_columns]

# For brand dropdown
brands = sorted(original_df['brand'].dropna().unique().tolist())

# For notable_effects dropdown (extract from one-hot columns)
effect_columns = [col for col in original_df.columns if col.startswith('notable_effects_')]
effects = [col.replace('notable_effects_', '') for col in effect_columns]

# === Define Widgets ===
brand_dropdown = widgets.Dropdown(options=brands, description='Brand:')
effect_dropdown = widgets.Dropdown(options=effects, description='Effect:')
skin_dropdown = widgets.Dropdown(options=['Oily', 'Dry', 'Normal'], description='Skin Type:')
price_input = widgets.BoundedIntText(value=100, min=10, max=1000, step=10, description='Price:')
submit_button = widgets.Button(description="Get Recommendation")
output = widgets.Output()

# === GPT prompt function ===
def ask_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content

# === Submit button logic ===
def on_submit(b):
    with output:
        output.clear_output()

        # Get user inputs
        brand = brand_dropdown.value
        effect = effect_dropdown.value
        skin = skin_dropdown.value
        price = price_input.value

        # === Create input for model ===
        input_dict = {
            'brand_' + brand: 1,
            'price_SAR': price,
            f'notable_effects_{effect}': 1,
            f'{skin}': 1
        }

        # Fill remaining model features with 0
        full_input = pd.DataFrame([{
            col: input_dict.get(col, 0) for col in model.feature_names_
        }])

        # Predict product type
        predicted_index = model.predict(full_input)[0].item()
        predicted_label = index_to_label[predicted_index]

        # === First Template ===
        prompt_1 = f"""
        You are a skincare advisor. The user has {skin} skin, wants {effect} results,
        prefers {brand} brand, and has a budget around {price} SAR.
        The recommended product type is: {predicted_label}.
        Please explain why this product type fits their needs in a friendly, informative way.
        """

        # === Second Template ===
        prompt_2 = f"""
        As a skincare consultant, analyze the user's profile:
        - Skin Type: {skin}
        - Concern: {effect}
        - Preferred Brand: {brand}
        - Budget: {price} SAR
        - Predicted Product Type: {predicted_label}
        Provide an explanation of why this product type is recommended,
        and how it works for their specific concern and skin type.
        """

        # GPT responses
        response_1 = ask_gpt(prompt_1)
        response_2 = ask_gpt(prompt_2)

        # === Display results ===
        display(HTML(f"<h4 style='color:green;'>Predicted Product Type: {predicted_label}</h4>"))
        display(HTML(f"<p><strong>GPT Advice 1 :</strong><br>{response_1}</p>"))
        display(HTML(f"<p><strong>GPT Advice 2 :</strong><br>{response_2}</p>"))


# Bind button
submit_button.on_click(on_submit)

# === Display UI ===
print("Fill in your details and click Submit:")
display(brand_dropdown, effect_dropdown, skin_dropdown, price_input, submit_button, output)


✅ Model Feature Names:
["brand_A'PIEU", 'brand_ACWELL', 'brand_AHC', 'brand_AIZEN', 'brand_ANESSA', 'brand_ARGAVELL', 'brand_ARIUL', 'brand_AVOSKIN', 'brand_AZARINE', 'brand_BANANA BOAT', 'brand_BANILA CO', 'brand_BE THE SKIN', 'brand_BENTON', 'brand_BHUMI', 'brand_BIO BEAUTY LAB', 'brand_BIO ESSENCE', 'brand_BIODERMA', 'brand_BIORE', 'brand_BIYU', 'brand_BLOOMKA', 'brand_BLP BEAUTY', 'brand_BREYLEE', 'brand_BY ECOM', 'brand_CARASUN', 'brand_CETAPHIL', 'brand_CHARIS', 'brand_CLINIQUE', 'brand_COSRX', 'brand_DEAR ME BEAUTY', 'brand_DERMALUZ', 'brand_DEW IT', 'brand_DR JART', 'brand_ELSHE SKIN', 'brand_ELVICTO', 'brand_EMINA', 'brand_ERHA', 'brand_ESSEN HERB', 'brand_ESTEE LAUDER', 'brand_ETUDE', 'brand_EUCALIE', 'brand_EVERPURE', 'brand_EVERSHINE', 'brand_EVERWHITE', 'brand_FIRST LAB', 'brand_FOR SKINS SAKE', 'brand_FRUDIA', 'brand_GARNIER', 'brand_GENTLE HOUR', 'brand_GLAMGLOW', 'brand_GLOWINC POTION', 'brand_HADA LABO', 'brand_HALE', 'brand_HANASUI', 'brand_HAPLE', 'brand_HARLETTE', '

Dropdown(description='Brand:', options=("A'PIEU", 'ACWELL', 'AHC', 'AIZEN', 'ANESSA', 'ARGAVELL', 'ARIUL', 'AV…

Dropdown(description='Effect:', options=('acne-free', 'acne-spot', 'anti-aging', 'balancing', 'black-spot', 'b…

Dropdown(description='Skin Type:', options=('Oily', 'Dry', 'Normal'), value='Oily')

BoundedIntText(value=100, description='Price:', max=1000, min=10, step=10)

Button(description='Get Recommendation', style=ButtonStyle())

Output()

### Prompt Comparison


### Chosen Prompt and Justification

### References

**[1]** IBM, "What is Generative AI?," IBM Think, [Online]. Available: https://www.ibm.com/think/topics/generative-ai. 

**[2]** Lighthouse Guild, "What is ChatGPT?," [Online]. Available: https://lighthouseguild.org/what-is-chat-gpt/.

**[3]** Harvard University Information Technology, "Writing Better Prompts for AI Tools," Harvard University IT, [Online]. Available: https://www.huit.harvard.edu/news/ai-prompts.
